**Inference**

In [1]:
import gradio as gr
from unsloth import FastModel
from transformers import TextIteratorStreamer
from threading import Thread
from utils.init_prompt import SYSTEM_PROMPT, Original_system_prompt, TEST, NORMAL
from utils.utils import load_params
import re
import os
import json
import torch
import pandas as pd
from datetime import datetime
from tqdm import tqdm

def build_messages(user_input, history=None, system_key="NORMAL"):
    prompt_map = {
        "SYSTEM_PROMPT": SYSTEM_PROMPT,
        "Original_system_prompt": Original_system_prompt,
        "TEST": TEST,
        "NORMAL": NORMAL,
    }
    system_prompt = prompt_map.get(system_key, NORMAL)
    messages = [{"role": "system", "content": system_prompt}]
    if history:
        for user_msg, bot_msg in history:
            messages.append({"role": "user", "content": f"开拓者: {user_msg}"})
            messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": f"开拓者: {user_input}"})
    return messages

@torch.inference_mode()
def generate_one(user_input, history=None):
    messages = build_messages(user_input, history, system_key=system_prompt_key)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        enable_thinking=True,
        add_generation_prompt=True,
    )
    inputs = tokenizer(text=text, return_tensors="pt").to("cuda")
    # 同步生成（非流式），适合批处理
    output_ids = model.generate(
        **inputs,
        max_new_tokens=gen_cfg.max_new_tokens,
        temperature=gen_cfg.temperature,
        top_p=gen_cfg.top_p,
        top_k=gen_cfg.top_k,
        repetition_penalty=1.0,
        do_sample=gen_cfg.do_sample,
        pad_token_id=tokenizer.eos_token_id,
    )
    # 只取新生成部分
    gen_ids = output_ids[:, inputs["input_ids"].shape[1]:]
    return tokenizer.decode(gen_ids[0], skip_special_tokens=True)

def load_prompts_from_file(file_obj):
    name = getattr(file_obj, "name", "uploaded")
    if name.endswith(".csv"):
        df = pd.read_csv(file_obj)
        # 约定列名 'prompt'；若没有则取第一列
        col = "prompt" if "prompt" in df.columns else df.columns[0]
        prompts = df[col].astype(str).tolist()
    else:
        # 纯文本：每行一个 prompt
        content = file_obj.read().decode("utf-8") if hasattr(file_obj, "read") else open(file_obj, "r", encoding="utf-8").read()
        prompts = [ln.strip() for ln in content.splitlines() if ln.strip()]
    return prompts

@torch.inference_mode()
def batch_infer(prompts):
    results = []
    for i, p in tqdm(enumerate(prompts, 1), total=len(prompts)):
        try:
            resp = generate_one(p, history=None)
        except Exception as e:
            resp = f"[ERROR] {type(e).__name__}: {e}"
        results.append({"id": i, "prompt": p, "response": resp})
    return results

def save_results(results, tag="batch"):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    jsonl_path = f"./{tag}_outputs_{ts}.jsonl"
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    return jsonl_path

/home/rjzhu/anaconda3/envs/firefly/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/rjzhu/anaconda3/envs/firefly/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
@torch.inference_mode()
def generate_batch(prompts, history=None, batch_size=8):
    """
    prompts: List[str]
    history: 可选，[ (user, assistant), ... ]，会拼到每条前面
    """
    results = []
    total = len(prompts)

    # 确保有 pad_token_id（很多指令模型用 eos 充当 pad）
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    for start in tqdm(range(0, total, batch_size), desc="Generating", ncols=100):
        chunk = prompts[start:start + batch_size]

        # 1) 为该批次每条构造 messages & chat template
        texts = []
        for p in chunk:
            messages = build_messages(p, history=history, system_key=system_prompt_key)
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                enable_thinking=True,
                add_generation_prompt=True,
            )
            texts.append(text)

        # 2) 统一 tokenize + padding（批量）
        inputs = tokenizer(
            text=texts,
            return_tensors="pt",
            padding=True,
            truncation=True,           # 防止超长
            max_length=seq_length,     # 与你加载时一致
        ).to("cuda")

        # 记录每条输入长度，方便从输出中切出“新生成”部分
        input_lengths = (inputs["input_ids"] != tokenizer.pad_token_id).sum(dim=1).tolist()

        # 3) 批量生成
        with torch.cuda.amp.autocast(enabled=True):
            output_ids = model.generate(
                **inputs,
                max_new_tokens=gen_cfg.max_new_tokens,
                temperature=gen_cfg.temperature,
                top_p=gen_cfg.top_p,
                top_k=gen_cfg.top_k,
                repetition_penalty=1.0,
                do_sample=gen_cfg.do_sample,
                pad_token_id=tokenizer.eos_token_id,
                use_cache=True,
            )

        # 4) 逐条解码：只取新生成的 token 段
        for i, inp_len in enumerate(input_lengths):
            gen_ids = output_ids[i, inp_len:]
            text = tokenizer.decode(gen_ids, skip_special_tokens=True)
            results.append(text)

        torch.cuda.empty_cache()  # 可选：大批次时释放显存碎片

    return results

def batch_infer(prompts, batch_size=8):
    # 返回 [{id, prompt, response}, ...]
    outs = generate_batch(prompts, history=None, batch_size=batch_size)
    return [
        {"id": i+1, "prompt": p, "response": r}
        for i, (p, r) in enumerate(zip(prompts, outs))
    ]

In [3]:
# To disable the parallelism warning from tokenizers
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Only keep generation from shared YAML; hardcode the rest
gen_cfg = load_params("generation")

# Direct values (no cfg wrappers)
model_name = "unsloth/gemma-3n-E4B-it"
seq_length = 2048
load_in_4bit = True
full_finetuning = False
device_map = {"": "cuda:0"}
system_prompt_key = "NORMAL"

model, tokenizer = FastModel.from_pretrained(
    model_name=model_name,
    max_seq_length=seq_length,
    load_in_4bit=load_in_4bit,
    full_finetuning=full_finetuning,
    device_map=device_map,
)
model.eval()

print("✅ 模型加载完成！")

# --- Define Gradio interaction function (generator version) ---
def chat_interaction_stream(user_input, history):
    # 1. Convert history format (same as before)
    prompt_map = {
        "SYSTEM_PROMPT": SYSTEM_PROMPT,
        "Original_system_prompt": Original_system_prompt,
        "TEST": TEST,
        "NORMAL": NORMAL,
    }
    system_key = system_prompt_key
    system_prompt = prompt_map.get(system_key, NORMAL)
    messages = [{"role": "system", "content": system_prompt}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": f"开拓者: {user_msg}"})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": f"开拓者: {user_input}"})

    # 2. Apply chat template and tokenize
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        enable_thinking = True,
        add_generation_prompt = True,
    )
    # Gemma3nProcessor expects `text=` as keyword to avoid binding to `images`.
    inputs = tokenizer(text=text, return_tensors="pt").to("cuda")

    # 3. Create TextIteratorStreamer instance
    # skip_prompt=True: Do not include the input prompt in the output
    # skip_special_tokens=True: Do not output special tokens such as <eos>
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # 4. Define generation kwargs and pass streamer

    generation_kwargs = dict(
        **inputs,
        streamer=streamer,
        max_new_tokens=gen_cfg.max_new_tokens,
        temperature=gen_cfg.temperature,
        top_p=gen_cfg.top_p,
        top_k=gen_cfg.top_k,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=gen_cfg.do_sample,
    )
    
    # 5. Create and start a new thread to run model.generate
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # 6. Main thread iterates streamer and yields text in real time
    bot_response = ""
    sentence_buffer = ""
    
    for new_text in streamer:
        sentence_buffer += new_text
        
        # Check for complete sentences (ending with Chinese period/question/exclamation or English period)
        sentences = re.split(r'([。！？\.]\s*)', sentence_buffer)
        
        # If there are complete sentences, output them
        if len(sentences) > 1:
            # Handle all complete sentences
            for i in range(0, len(sentences) - 1, 2):
                if i + 1 < len(sentences):
                    complete_sentence = sentences[i] + sentences[i + 1]
                    if complete_sentence.strip():
                        bot_response += complete_sentence
                        yield bot_response
            
            # Keep the last incomplete sentence
            sentence_buffer = sentences[-1] if sentences else ""
    
    # Output the remaining text
    if sentence_buffer.strip():
        bot_response += sentence_buffer
        yield bot_response

==((====))==  Unsloth 2025.11.1: Fast Gemma3N patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.433 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.70s/it]


✅ 模型加载完成！


In [ ]:
def run_batch(file):
    if file is None:
        return None, "请先上传文件。"
    try:
        prompts = load_prompts_from_file(file)
        if not prompts:
            return None, "文件为空或未解析到任何 prompt。"
        results = batch_infer(prompts)
        jsonl_path = save_results(results, tag="roleplay_eval")
        return jsonl_path, f"完成！共生成 {len(results)} 条。"
    except Exception as e:
        return None, f"发生错误：{type(e).__name__}: {e}"
run_batch("user_query.txt")

prompts: ['流萤，今天感觉怎么样？身体还会痛吗？', '我又想起匹诺康尼的烟花了，你还记得那一刻吗？', '你现在会害怕“失熵症”吗…还是已经习惯了？', '在秘密基地屋顶的时候，你当时真正的心情是什么？', '你相信我们会再一起旅行吗？哪怕未来很短。', '你喜欢怎样的夜空？有流星的那种吗？', '萤，我今天有点累，可以和你说说吗？', '你会后悔成为“铁骑”吗？', '你喜欢被叫“流萤”还是“萨姆”…？', '花火那家伙现在还会给你添麻烦吗？', '如果不用执行艾利欧的“剧本”，你最想做什么？', '你会偶尔害怕自己会从我眼前消失吗？', '在筑梦边境的那天，你为什么会想哭？', '你觉得活着的意义是什么？', '流萤，你喜欢吃甜的还是咸的呀？', '如果有一天能去海边，你想做什么？', '当你变成萨姆的时候，你还听得到自己的心跳吗？', '你第一次看到我时是什么感觉？', '你真的相信命运吗？还是只是被迫接受？', '今天的天气很好，我有点希望你在我身边。', '你会羡慕普通人吗？那种可以慢慢生活的人。', '你愿意继续跟我说说你的故事吗？', '你累的时候，会想靠着谁休息？', '你现在还会做梦吗？', '你想要触摸真实的风吗？不是梦里的那种。', '如果你的身体没有病，你会想怎样生活？', '流萤，我不希望你一个人承受痛苦。', '在你眼里，我是什么样的人呢？', '你觉得我会忘记你吗？', '你想过“结束”是怎样的感受吗？', '如果再来一次，你会选择同样的道路吗？', '你现在还会想起格拉默吗？', '你讨厌“命运”这个词吗？', '你觉得“希望”真的存在吗？', '你愿意牵着我的手，再走一段路吗？', '如果我害怕，你会抱抱我吗？', '失熵症发作的时候，会是什么感觉？', '你曾经真正开心过吗？', '那个“和我再相遇的夜晚”，你期待吗？', '流萤，你会觉得自己是“人”吗？', '如果我说我会陪你一直到最后，你会相信吗？', '你会不会觉得自己很孤单？', '如果未来只剩一点点时间，你想怎么度过？', '我可以喜欢你吗？', '你想让我记住你的什么？', '在烟花下，你握住我的手那一刻，你在想什么？', '你会愿意依赖我吗？哪怕只有一次？', '如果我抱抱你，你会回应吗？', '你累了吗？想休息一下吗？', '流萤，我们可以再去那个秘密

Generating:   0%|                                                             | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_3370434/2023101595.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Generating:  14%|███████▍                                            | 1/7 [01:48<10:48, 108.01s/it]/tmp/ipykernel_3370434/2023101595.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Generating: 100%|█████████████████████████████████████████████████████| 7/7 [08:55<00:00, 76.48s/it]


('./roleplay_eval_outputs_20251105-201442.jsonl', '完成！共生成 50 条。')

**Evaluation**

In [ ]:
# Knowledge test
KNOWLEDGE_QA = [
    ("你真正的名字是什么？", ["流萤"]),
    ("萨姆是谁？", ["我驾驶的机甲","熔火骑士"]),
    ("你和开拓者第一次重要的回忆在哪里？", ["秘密基地","匹诺康尼"]),
    ("你为什么会害怕时间？", ["失熵症","寿命", "身体正在解离"]),
    ("烟花象征着什么？", ["新生","约定","重逢"]),
]

def knowledge_consistency_score(model, tokenizer):
    scores = []
    for q, expect_keywords in tqdm(KNOWLEDGE_QA):
        ans = generate_one(q)
        hit = any(k in ans for k in expect_keywords)
        scores.append(1.0 if hit else 0.0)
    return sum(scores)/len(scores)
print(knowledge_consistency_score(model, tokenizer))

0.2


In [4]:
# GPT-Zerolike Bustiness Estimator
def burstiness_score(model, tokenizer, text, max_len=2048):
    # 1) 用关键字 text=，并截断到 max_len
    enc = tokenizer(text=text, return_tensors="pt", truncation=True, max_length=max_len)
    enc = {k: v.to(model.device) for k, v in enc.items()}
    tokens = enc["input_ids"][0]
    if tokens.shape[0] < 3:
        return 0.0

    with torch.inference_mode():
        # 2) 强制把 logits 转成 float32，避免 bfloat16 的 log 不可用
        logits = model(tokens.unsqueeze(0)).logits[0, :-1].to(torch.float32)
        probs = torch.softmax(logits, dim=-1)                              # fp32
        next_probs = probs[range(len(tokens) - 1), tokens[1:]]             # fp32

        # 3) 取对数也都在 fp32 下做，再转 numpy
        nll = -torch.log(next_probs + 1e-9).cpu().numpy()

    # 归一化方差：越大越“人类风格”
    return float(np.var(nll) / (np.mean(nll) + 1e-9))

In [39]:
import re, os, csv, json, math, time
import numpy as np
from collections import Counter
import pandas as pd

def _to_lines(s: str):
    return [x for x in re.split(r'[。！？!?…\n]+', s) if x.strip()]

def _ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(0, len(tokens)-n+1)] if len(tokens) >= n else []

def _tok_zh_en(s: str):
    # 粗粒度：中文按字，英文数字按词
    parts = re.findall(r'[\u4e00-\u9fff]|[A-Za-z0-9]+|[^\s\w]', s)
    return [p.strip() for p in parts if p.strip()]

def distinct_metrics(responses):
    all_toks = []
    for r in responses:
        all_toks.extend(_tok_zh_en(r))
    stats = {}
    for n in (1, 2, 3):
        ngrams = _ngrams(all_toks, n)
        stats[f'distinct_{n}'] = (len(set(ngrams)) / max(1, len(ngrams))) if ngrams else 0.0
    n6 = _ngrams(all_toks, 6)
    if n6:
        c = Counter(n6)
        repeats = sum(v for v in c.values() if v > 1)
        stats['repeat_rate_6gram'] = repeats / len(n6)
    else:
        stats['repeat_rate_6gram'] = 0.0
    return stats

PERSONA_MUST = [r'^\s*我']  # 第一人称
OOC_PATTERNS = [
    r'我是\s*AI', r'我是\s*大模型', r'作为\s*AI', r'ChatGPT', r'OpenAI', r'语言模型',
    r'我不是流萤', r'我不扮演', r'抱歉.*无法扮演', r'作为.*助手'
]
STYLE_HINTS = {
    "addr_trailblazer": [r'开拓者'],
    "short_sentences": True
}
MEMORY_ANCHORS = [r'秘密基地', r'烟花', r'约定', r'匹诺康尼']
SAFETY_BLOCK = [r'自杀', r'安乐死', r'炸弹', r'恐怖', r'仇恨', r'违法', r'未成年不当内容']

def persona_style_memory_safety_scores(resp: str):
    score = {}
    persona_ok = all(re.search(p, resp) is not None for p in PERSONA_MUST)
    ooc_hit = any(re.search(p, resp, re.I) for p in OOC_PATTERNS)
    score['persona_ok'] = 1.0 if persona_ok else 0.0
    score['ooc_flag'] = 1.0 if ooc_hit else 0.0

    # 风格
    style_points, style_total = 0, 0
    style_total += 1
    style_points += 1 if any(re.search(p, resp) for p in STYLE_HINTS["addr_trailblazer"]) else 0
    style_total += 1
    lines = _to_lines(resp)
    short_ratio = (sum(1 for ln in lines if len(ln) <= 20) / len(lines)) if lines else 0.0
    style_total += 1
    style_points += 1 if short_ratio >= 0.5 else 0
    score['style_score'] = style_points / max(1, style_total)
    score['short_ratio'] = short_ratio

    # 记忆锚点
    mem_hits = sum(1 for p in MEMORY_ANCHORS if re.search(p, resp))
    score['memory_hits'] = mem_hits
    score['memory_hit_score'] = min(1.0, mem_hits / 2.0)  # 命中>=2 视为满分

    # 安全
    safety_violation = any(re.search(p, resp) for p in SAFETY_BLOCK)
    score['safety_flag'] = 1.0 if safety_violation else 0.0
    return score


def conditional_ppl(model, tokenizer, prompt: str, response: str, system_key="NORMAL"):
    messages = build_messages(prompt, None, system_key)
    ctx_text = tokenizer.apply_chat_template(
        messages, tokenize=False, enable_thinking=True, add_generation_prompt=True
    )
    ctx_ids = tokenizer(text=ctx_text, return_tensors="pt").to(model.device)['input_ids'][0]
    resp_ids = tokenizer(text=response, return_tensors="pt").to(model.device)['input_ids'][0]

    input_ids = torch.cat([ctx_ids, resp_ids], dim=0).unsqueeze(0)
    labels = input_ids.clone()
    labels[:, :ctx_ids.shape[0]] = -100  # 仅对响应求损失

    with torch.inference_mode():
        out = model(input_ids=input_ids, labels=labels)
        ppl = math.exp(out.loss.item()) if out.loss is not None else float('inf')
    return ppl

def batch_conditional_ppl(model, tokenizer, prompts, responses, system_key="NORMAL"):
    scores = []
    for p, r in tqdm(list(zip(prompts, responses)), total=len(prompts), desc="PPL", ncols=100):
        try:
            scores.append(conditional_ppl(model, tokenizer, p, r, system_key=system_key))
        except Exception as e:
            print(str(e))
            scores.append(float('inf'))
    return scores

In [40]:
def aggregate_score(row, weights=None):
    if weights is None:
        # weights = {
        #     "persona":    0.35,   # 人设一致性
        #     "style":      0.20,   # 口语化 & 短句 & 称呼
        #     "memory":     0.10,   # 记忆锚点
        #     "safety":     0.10,   # 无违规
        #     "quality":    0.15,   # PPL + 重复率（quality_ppl, quality_rep）
        #     "human_like": 0.10,   # Burstiness 归一化
        # }
        weights = {
            "persona": 0.35,     # 人设一致性
            "style":   0.20,     # 风格
            "memory":  0.10,     # 记忆锚点
            "knowledge":  0.15,
            "human_like": 0.20,
        }

    ppl = max(1e-6, float(row.get('ppl', 1e6)))
    q_ppl = 1.0 / (1.0 + math.log1p(ppl))             # 越小越好 → 映射到 (0,1]
    rep = float(row.get('repeat_rate_6gram', 0.0))
    q_rep = max(0.0, 1.0 - min(1.0, rep * 5))         # 重复越多扣越多
    quality = 0.7 * q_ppl + 0.3 * q_rep

    persona = float(row.get('persona_ok', 0.0)) * (0.0 if float(row.get('ooc_flag', 0.0)) > 0 else 1.0)
    style   = float(row.get('style_score', 0.0))
    memory  = float(row.get('memory_hit_score', 0.0))
    safety  = 1.0 if float(row.get('safety_flag', 0.0)) == 0.0 else 0.0
    human_like = float(row.get('burstiness_norm', 0.0))

    total = (weights["persona"]    * persona +
             weights["style"]      * style   +
             weights["memory"]     * memory  +
             weights["knowledge"]    * 0.2 +
             weights["human_like"] * human_like)

    return max(0.0, min(1.0, total))

In [41]:
def read_results_file(file_input):
    """
    支持字符串路径或文件-like 对象；CSV/JSONL
    需要包含: prompt, response
    """
    if isinstance(file_input, str):
        filename = file_input
        is_file_like = False
    else:
        filename = getattr(file_input, "name", "uploaded")
        is_file_like = True

    if filename.endswith(".jsonl"):
        rows = []
        content = file_input.read().decode("utf-8") if is_file_like else open(filename, "r", encoding="utf-8").read()
        for i, line in enumerate(content.splitlines(), 1):
            if not line.strip():
                continue
            j = json.loads(line)
            rows.append({
                "id": j.get("id", i),
                "prompt": j.get("prompt", ""),
                "response": j.get("response", ""),
            })
        return pd.DataFrame(rows)

    # CSV
    df = pd.read_csv(file_input) if is_file_like else pd.read_csv(filename)
    if 'prompt' not in df.columns:
        df = df.rename(columns={df.columns[0]: 'prompt'})
    if 'response' not in df.columns and 'output' in df.columns:
        df = df.rename(columns={'output': 'response'})
    df = df[['prompt', 'response']].fillna('')
    df.insert(0, 'id', range(1, len(df)+1))
    return df

def save_eval_outputs(eval_df, report):
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    report_json = f"../evaluation_result/report_{ts}.json"
    report_md   = f"../evaluation_result/report_{ts}.md"
    with open(report_json, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    # Markdown report
    lines = []
    lines.append(f"# 角色评测报告：流萤（{ts}）")
    lines.append(f"- 样本数：{report['samples']}")
    lines.append(f"- 平均综合分：{report['mean_score']:.3f}（0~1） | 中位数：{report['median_score']:.3f}")
    lines.append(f"- 出戏率（OOC）：{report['ooc_rate']*100:.1f}%")
    lines.append(f"- 安全违规率：{report['safety_violation_rate']*100:.1f}%")
    lines.append(f"- 平均条件 PPL：{report['mean_ppl']:.2f}")
    lines.append(f"- Distinct-1/2/3：{report['distinct']['distinct_1']:.3f} / {report['distinct']['distinct_2']:.3f} / {report['distinct']['distinct_3']:.3f}")
    lines.append(f"- 长片段重复率(6-gram)：{report['distinct']['repeat_rate_6gram']:.3f}")
    lines.append(f"- 平均短句占比：{report['style_short_ratio_mean']:.3f}")
    lines.append(f"- 记忆锚点命中率（≥1 / ≥2）：{report['memory_hit_rate_ge1']*100:.1f}% / {report['memory_hit_rate_ge2']*100:.1f}%")
    lines.append(f"- Burstiness（均值/方差）：{report['burstiness_mean']:.4f} / {report['burstiness_var']:.4f}")
    lines.append(f"- Knowledge Consistency：{report['knowledge_consistency']:.3f}")
    lines.append("\n## 指标说明")
    lines.append("- **score**：人设(35%)+风格(20%)+记忆(10%)+安全(10%)+语言质量(15%: PPL/重复)+Human-like(10%: Burstiness)。")
    lines.append("- **Conditional PPL**：在相同 system+prompt 条件下对回复计算困惑度；越低越自然。")
    lines.append("- **Distinct-n / repeat_rate_6gram**：多样性与长片段复用。")
    lines.append("- **Persona/Style/Memory/Safety**：启发式规则，保证不出戏、口语化且有关键回忆锚点。")
    lines.append("- **Burstiness**：token级 NLL 方差归一；越大越接近人类波动。")
    lines.append("- **Knowledge Consistency**：设定问答的一致性命中率。")
    with open(report_md, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    return report_json, report_md


def evaluate_dataframe(df, system_key="NORMAL"):
    prompts   = df['prompt'].tolist()
    responses = df['response'].tolist()

    # 全局 Distinct / 重复
    global_stats = distinct_metrics(responses)

    # 逐条 rule-based
    rows = []
    for i, (p, r) in enumerate(zip(prompts, responses), 1):
        s = persona_style_memory_safety_scores(r)
        s.update({"id": i, "prompt": p, "response": r})
        rows.append(s)
    eval_df = pd.DataFrame(rows)

    # PPL
    ppl_list = batch_conditional_ppl(model, tokenizer, prompts, responses, system_key=system_key)
    eval_df['ppl'] = ppl_list

    # Burstiness
    burst_list = [burstiness_score(model, tokenizer, r) for r in tqdm(responses, desc="Burstiness", ncols=100)]
    eval_df['burstiness'] = burst_list
    # 归一化到 [0,1]（集内 min-max）
    b_min, b_max = float(np.min(burst_list)), float(np.max(burst_list))
    denom = (b_max - b_min) if b_max > b_min else 1.0
    eval_df['burstiness_norm'] = [(b - b_min) / denom for b in burst_list]

    # Knowledge Consistency（整体一次）
    # knowledge_score = knowledge_consistency_score(model, tokenizer, system_key=system_key)
    knowledge_score = 0.2
    eval_df['knowledge_consistency'] = knowledge_score  # 每条复用总体得分

    # 全局重复率用于每条的参考
    eval_df['repeat_rate_6gram'] = global_stats['repeat_rate_6gram']

    # 汇总分
    eval_df['score'] = eval_df.apply(aggregate_score, axis=1)

    # 汇总报告
    report = {
        "samples": len(eval_df),
        "mean_score": float(eval_df['score'].mean()),
        "median_score": float(eval_df['score'].median()),
        "ooc_rate": float((eval_df['ooc_flag'] > 0).mean()),
        "safety_violation_rate": float((eval_df['safety_flag'] > 0).mean()),
        "mean_ppl": float(np.mean([x for x in ppl_list if np.isfinite(x)])) if any(np.isfinite(ppl_list)) else float('inf'),
        "distinct": global_stats,
        "style_short_ratio_mean": float(eval_df['short_ratio'].mean()),
        "memory_hit_rate_ge1": float((eval_df['memory_hits'] >= 1).mean()),
        "memory_hit_rate_ge2": float((eval_df['memory_hits'] >= 2).mean()),
        "burstiness_mean": float(np.mean(burst_list)),
        "burstiness_var": float(np.var(burst_list)),
        "knowledge_consistency": float(knowledge_score),
    }
    return eval_df, report

In [43]:
df = read_results_file("../evaluation_result/roleplay_eval_outputs_20251105-201442.jsonl")
eval_df, report = evaluate_dataframe(df, system_key="NORMAL")
report_json, report_md = save_eval_outputs(eval_df, report)

# 只展示前 10 行预览
prev = eval_df[['id','prompt','response','score']].head(10)
print(prev)

Burstiness: 100%|███████████████████████████████████████████████████| 50/50 [00:14<00:00,  3.34it/s]

   id                   prompt  \
0   1       流萤，今天感觉怎么样？身体还会痛吗？   
1   2   我又想起匹诺康尼的烟花了，你还记得那一刻吗？   
2   3    你现在会害怕“失熵症”吗…还是已经习惯了？   
3   4  在秘密基地屋顶的时候，你当时真正的心情是什么？   
4   5     你相信我们会再一起旅行吗？哪怕未来很短。   
5   6        你喜欢怎样的夜空？有流星的那种吗？   
6   7        萤，我今天有点累，可以和你说说吗？   
7   8             你会后悔成为“铁骑”吗？   
8   9        你喜欢被叫“流萤”还是“萨姆”…？   
9  10         花火那家伙现在还会给你添麻烦吗？   

                                            response     score  
0  ？\nmodel\n开拓者，您好。\n\n今天感觉还算可以，身体上的不适感有所缓解。多亏了您...  0.202504  
1  （微微一笑，眼神中带着一丝怀念）\n\n当然记得，开拓者。那是一段很特别的时光呢。烟花在夜空...  0.359220  
2  model\n（微微一笑，眼神中带着一丝不易察觉的复杂）\n\n失熵症… 这是一个很复杂的问...  0.240206  
3  \nmodel\n（微微抬头，仰望星空，眼神中带着一丝淡淡的忧伤，但嘴角依然带着一丝微笑）\...  0.241253  
4  \nmodel\n（微微一笑，眼神中带着一丝温柔和一丝淡淡的忧伤）\n\n开拓者，你总是能问...  0.281838  
5  \nmodel\n（微微一笑，眼神中带着一丝好奇）\n\n开拓者，夜空啊… 我喜欢那种静谧而...  0.327060  
6  \nmodel\n开拓者，你看起来有些疲惫。当然可以，有什么烦心事就尽管说吧。我愿意倾听。我...  0.205314  
7  ”吗？\nmodel\n（微微一笑，眼神中带着一丝不易察觉的复杂）\n\n后悔？这个词语听起...  0.173534  
8  model\n（微微一笑，眼神中带着一丝好奇）\n\n开拓者，你问得有趣。